### # Appendix : PyTorch crash course

In [1]:
import torch

torch.__version__

'2.5.1+cpu'

In [2]:
torch.cuda.is_available()

False

### A.2 Understanding tensors

In [3]:
# Creating PyTorch tensors

tensor0d = torch.tensor(1,)
tensor1d = torch.tensor([1, 2, 3])
tensor2d = torch.tensor([
    [1, 2], 
    [3, 4]
])
tensor3d = torch.tensor([
    [[1, 2], [3, 4]],
    [[5, 6], [7, 8]],
    [[5, 6], [7, 8]]
]) 

In [4]:
print(tensor2d)
print(tensor2d.ndim)
print(tensor3d.shape)

tensor([[1, 2],
        [3, 4]])
2
torch.Size([3, 2, 2])


In [5]:
# Type conversion
floatvec = tensor1d.to(torch.float32)
print(tensor1d)
print(floatvec)

tensor([1, 2, 3])
tensor([1., 2., 3.])


In [6]:
print(tensor2d.reshape(1, 4))
# Or 
print(tensor2d.view(1, 4))

tensor([[1, 2, 3, 4]])
tensor([[1, 2, 3, 4]])


In [7]:
print(tensor2d.T)

tensor([[1, 3],
        [2, 4]])


In [8]:
# multiply two matrice

print(tensor2d.matmul(tensor2d.T))
# Or 
print(tensor2d @ tensor2d.T)

tensor([[ 5, 11],
        [11, 25]])
tensor([[ 5, 11],
        [11, 25]])


### A.3 Seeing models as computation graphs

In [9]:
# Listing A.2 A logistic regression forward pass

y = torch.tensor([1.0]) # True label
x1 = torch.tensor([1.1]) # Input feature
w1 = torch.tensor([2.2]) # Weight parameter
b = torch.tensor([0.0]) # Bias unit

z = x1 * w1 + b # Net input
a = torch.sigmoid(z) # Activation and output

loss = torch.nn.functional.binary_cross_entropy(a, y)
print(a)
print(loss)

tensor([0.9183])
tensor(0.0852)


### A.4 Automatic differentiation made easy

In [10]:
# Listing A.3 Computing gradients via autograd

from torch.autograd import grad


y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)

z = x1 * w1 + b
a = torch.sigmoid(z)

loss = torch.nn.functional.binary_cross_entropy(a, y)

# to calculate grads we can use
grad_L_w1 = grad(loss, w1, retain_graph=True) # dL/dw1
grad_L_b = grad(loss, b, retain_graph=True) # dL/db
print(grad_L_w1)
print(grad_L_b)

# Or
loss.backward()
print("\n")
print(w1.grad)
print(b.grad)

(tensor([-0.0898]),)
(tensor([-0.0817]),)


tensor([-0.0898])
tensor([-0.0817])


### A.5 Implementing multilayer neural networks

In [11]:
# Listing A.4 A multilayer perceptron with two hidden layers 

class NeuralNetwork(torch.nn.Module):

    def __init__(self, num_inputs, num_outputs):
        super().__init__()
        self.layers = torch.nn.Sequential(
            # 1st hidden layer
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),
            # 2nd hidden layer
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),
            # output layer
            torch.nn.Linear(20, num_outputs),
        )
    
    def forward(self, x):
        """calculating output tensors from input tensors"""
        logits = self.layers(x) # The outputs of the last layer
        return logits 

In [12]:
model = NeuralNetwork(50, 3)
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [13]:
50*30 + 30*20 + 20*3 + (30+20+3)

2213

In [14]:
# check the total number of trainable parameters
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total number of trainable model parameters:", num_params)

Total number of trainable model parameters: 2213


In [15]:
print(model.layers[0].weight)
print(model.layers[0].weight.shape)


Parameter containing:
tensor([[ 0.0253,  0.0321,  0.0554,  ...,  0.0457,  0.0702, -0.1286],
        [-0.0452, -0.0277, -0.0195,  ...,  0.0383, -0.0454, -0.1117],
        [-0.0557, -0.0645, -0.0635,  ...,  0.0202,  0.0115,  0.0521],
        ...,
        [ 0.0393,  0.0245, -0.1146,  ...,  0.0792, -0.1410, -0.1397],
        [ 0.1006, -0.0154,  0.0453,  ...,  0.1039,  0.1112,  0.0772],
        [-0.0862,  0.1207, -0.0366,  ...,  0.0324, -0.1381, -0.0724]],
       requires_grad=True)
torch.Size([30, 50])


In [16]:
print(model.layers[0].bias.shape)
print(model.layers[2].bias.shape)
print(model.layers[4].bias.shape)

torch.Size([30])
torch.Size([20])
torch.Size([3])


In [17]:
torch.manual_seed(123)
model = NeuralNetwork(50, 3)
print(model.layers[0].weight)

Parameter containing:
tensor([[-0.0577,  0.0047, -0.0702,  ...,  0.0222,  0.1260,  0.0865],
        [ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        ...,
        [-0.0787,  0.1259,  0.0803,  ...,  0.1218,  0.1303, -0.1351],
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509]],
       requires_grad=True)


In [18]:
torch.manual_seed(123)

X = torch.rand((1, 50))
out = model(X)

print(X.shape)
print(out)

# Or
with torch.no_grad():
    out = model(X)
print(out)

torch.Size([1, 50])
tensor([[-0.1262,  0.1080, -0.1792]], grad_fn=<AddmmBackward0>)
tensor([[-0.1262,  0.1080, -0.1792]])


In [19]:
# compute class-membership probabilities for our predictions
with torch.no_grad():
    out = torch.softmax(model(X), dim=1)
print(out)

tensor([[0.3113, 0.3934, 0.2952]])


### A.6 Setting up efficient data loaders
- PyTorch implements a Dataset and a DataLoader class. The Dataset class is used to instantiate objects that define how each data record is loaded. The DataLoader handles how the data is shuffled and assembled into batches

- we will implement a custom Dataset class, which we will use to create a training and a test dataset that we’ll then use to create the data loaders

In [20]:
# Listing A.5 Creating a small toy dataset 

X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])
y_train = torch.tensor([0, 0, 0, 1, 1])

X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6],
])
y_test = torch.tensor([0, 1])


In [21]:
# Listing A.6 Defining a custom Dataset class 

from torch.utils.data import Dataset

class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y

    def __getitem__(self, index):
        """Instructions for retrieving exactly one data record and the corresponding label"""
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y
    
    def __len__(self):
        return self.labels.shape[0]

In [22]:
train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

for i, t in train_ds:
    print(i, t)

tensor([-1.2000,  3.1000]) tensor(0)
tensor([-0.9000,  2.9000]) tensor(0)
tensor([-0.5000,  2.6000]) tensor(0)
tensor([ 2.3000, -1.1000]) tensor(1)
tensor([ 2.7000, -1.5000]) tensor(1)


In [23]:
# Listing A.7 Instantiating data loaders 

from torch.utils.data import DataLoader

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0
)

In [24]:
for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

Batch 1: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 2: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])


### A.7 A typical training loop

In [25]:
# Listing A.9 Neural network training in PyTorch 

import torch.nn.functional as F

torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)

optimizer = torch.optim.SGD(model.parameters(), lr=0.5)
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):
        logits = model(features)

        loss = F.cross_entropy(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        ### LOGGING
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
        f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
        f" | Train Loss: {loss:.2f}")
        
    model.eval()
 # Insert optional model evaluation code

Epoch: 001/003 | Batch 000/002 | Train Loss: 0.75
Epoch: 001/003 | Batch 001/002 | Train Loss: 0.65
Epoch: 002/003 | Batch 000/002 | Train Loss: 0.44
Epoch: 002/003 | Batch 001/002 | Train Loss: 0.13
Epoch: 003/003 | Batch 000/002 | Train Loss: 0.03
Epoch: 003/003 | Batch 001/002 | Train Loss: 0.00


In [26]:
# After we have trained the model, we can use it to make predictions:

model.eval()
with torch.no_grad():
    outputs = model(X_train)
print(outputs)

tensor([[ 2.8569, -4.1618],
        [ 2.5382, -3.7548],
        [ 2.0944, -3.1820],
        [-1.4814,  1.4816],
        [-1.7176,  1.7342]])


In [27]:
torch.set_printoptions(sci_mode=False)
probas = torch.softmax(outputs, dim=1)
print(probas)

tensor([[    0.9991,     0.0009],
        [    0.9982,     0.0018],
        [    0.9949,     0.0051],
        [    0.0491,     0.9509],
        [    0.0307,     0.9693]])


In [28]:
predictions = torch.argmax(probas, dim=1)
print(predictions)


tensor([0, 0, 0, 1, 1])


In [29]:
predictions == y_train


tensor([True, True, True, True, True])

In [30]:
torch.sum(predictions == y_train)

tensor(5)

In [31]:
# Listing A.10 A function to compute the prediction accuracy 

def compute_accuracy(model, dataloader):
    model = model.eval()
    correct = 0.0
    total_examples = 0

    for idx, (features, labels) in enumerate(dataloader):
        with torch.no_grad():
            logits = model(features)

        predictions = torch.argmax(logits, dim=1)
        compare = labels == predictions
        correct += torch.sum(compare)
        total_examples += len(compare)
        
    return (correct / total_examples).item()

In [32]:
print(compute_accuracy(model, train_loader))
print(compute_accuracy(model, test_loader))

1.0
1.0


### A.8 Saving and loading models


In [33]:
torch.save(model.state_dict(), "save_model.pth")

In [34]:
model

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=2, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=2, bias=True)
  )
)

In [35]:
# Load model
model = NeuralNetwork(2, 2)
model.load_state_dict(torch.load("save_model.pth"))

C:\Users\pc\AppData\Local\Temp\ipykernel_11812\1471068070.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("save_model.pth"))


<All keys matched successfully>

In [36]:
print(torch.cuda.is_available())


False
